SQuAD 2.0 Exploratory Data Analysis

In [ ]:
# pip install huggingface datasets
!pip install datasets

     |████████████████████████████████| 194kB 5.3MB/s 
     |████████████████████████████████| 245kB 9.1MB/s 
     |████████████████████████████████| 112kB 9.7MB/s 


In [ ]:
# import load_dataset
from datasets import load_dataset

# load squad 1.0 dataset
dataset = load_dataset("squad_v2")

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/9cac55034b086140f0649ecb5c604d09d7da2f2f5b73a90caa2e2bcc1f5cac09. Subsequent calls will reuse this data.


In [ ]:
# view dataset 
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
import json
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
# set json data to dataframe
train_df = pd.json_normalize(dataset['train'], errors='ignore')
validation_df = pd.json_normalize(dataset['validation'], errors='ignore')

In [ ]:
train_df.head()

,context,id,question,title,answers.answer_start,answers.text
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé,[269],[in the late 1990s]
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé,[207],[singing and dancing]
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé,[526],[2003]
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé,[166],"[Houston, Texas]"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé,[276],[late 1990s]


Statistics for SQuAD dataset

In [ ]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# create column for tokenized context
train_df["context_tokens"] = train_df['context'].apply(word_tokenize)

# create column for number of context tokens
train_df["num_c_tokens"] = [len(token) for token in train_df["context_tokens"]]

In [ ]:
# create column for tokenized question
train_df["question_tokens"] = train_df['question'].apply(word_tokenize)

# create column for number of question tokens
train_df["num_q_tokens"] = [len(token) for token in train_df["question_tokens"]]

In [ ]:
train_df.head()

,context,id,question,title,answers.answer_start,answers.text,context_tokens,num_c_tokens,question_tokens,num_q_tokens
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé,[269],[in the late 1990s],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[When, did, Beyonce, start, becoming, popular, ?]",7
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé,[207],[singing and dancing],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[What, areas, did, Beyonce, compete, in, when,...",12
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé,[526],[2003],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[When, did, Beyonce, leave, Destiny, 's, Child...",13
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé,[166],"[Houston, Texas]","[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[In, what, city, and, state, did, Beyonce, gro...",10
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé,[276],[late 1990s],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[In, which, decade, did, Beyonce, become, famo...",8


In [ ]:
# number of questions
n_question = len(train_df['context'])
print("Number of questions :", n_question)

Number of questions : 130319


In [ ]:
# number of unique contexts
unique_contexts = len(pd.unique(train_df['context']))
print("Number of unique contexts :", unique_contexts)

Number of unique contexts : 19029


In [ ]:
# average number of tokens per question
q_avg = train_df['num_q_tokens'].mean(axis = 0) 
print("Average number of tokens per question :", q_avg)

Average number of tokens per question : 11.106707387257423


In [ ]:
# average number of tokens per context
c_unique = train_df.drop_duplicates(subset = ['context'])
c_avg = c_unique['num_c_tokens'].mean(axis = 0) 
print("Average number of tokens per context :", c_avg)

Average number of tokens per context : 134.1707393977613


In [ ]:
# get the number of answerable questions
drop_no_answer = train_df[~train_df['answers.answer_start'].str.len().eq(0)]
num_ans_question = len(drop_no_answer['context'])
per_ans_question = (num_ans_question / n_question) * 100
print("Number of answerable questions :", num_ans_question)
print("Percentage of answerable questions :", per_ans_question)

Number of answerable questions : 86821
Percentage of answerable questions : 66.62190471074824


In [ ]:
# convert squad data back to seperate jsonl files

In [ ]:
# function to combine the start token and answer into dictionary
def to_dict(answer_start, answer): 
  dict = {}
  dict['answer_start'] = answer_start
  dict['text'] = answer
  return dict

In [ ]:
# run function
train_df['answers'] = train_df.apply(lambda x:to_dict(x['answers.answer_start'],x['answers.text']), axis=1)
validation_df['answers'] = validation_df.apply(lambda x:to_dict(x['answers.answer_start'],x['answers.text']), axis=1)

In [ ]:
pd.set_option('display.max_colwidth', 50)
train_df.head(100000)

,context,id,question,title,answers.answer_start,answers.text,context_tokens,num_c_tokens,question_tokens,num_q_tokens,answers
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé,[269],[in the late 1990s],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[When, did, Beyonce, start, becoming, popular, ?]",7,"{'answer_start': [269], 'text': ['in the late ..."
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé,[207],[singing and dancing],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[What, areas, did, Beyonce, compete, in, when,...",12,"{'answer_start': [207], 'text': ['singing and ..."
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé,[526],[2003],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[When, did, Beyonce, leave, Destiny, 's, Child...",13,"{'answer_start': [526], 'text': ['2003']}"
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé,[166],"[Houston, Texas]","[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[In, what, city, and, state, did, Beyonce, gro...",10,"{'answer_start': [166], 'text': ['Houston, Tex..."
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé,[276],[late 1990s],"[Beyoncé, Giselle, Knowles-Carter, (, /biːˈjɒn...",139,"[In, which, decade, did, Beyonce, become, famo...",8,"{'answer_start': [276], 'text': ['late 1990s']}"
...,...,...,...,...,...,...,...,...,...,...,...
99995,Secondary education in the United States did n...,5acd74fd07355d001abf4325,Why wasn't secondary school introduced in the ...,Education,[],[],"[Secondary, education, in, the, United, States...",93,"[Why, was, n't, secondary, school, introduced,...",10,"{'answer_start': [], 'text': []}"
99996,Secondary education in the United States did n...,5acd74fd07355d001abf4326,Who didn't benefit from secondary schools?,Education,[],[],"[Secondary, education, in, the, United, States...",93,"[Who, did, n't, benefit, from, secondary, scho...",8,"{'answer_start': [], 'text': []}"
99997,Secondary education in the United States did n...,5acd74fd07355d001abf4327,Why were high schools not created?,Education,[],[],"[Secondary, education, in, the, United, States...",93,"[Why, were, high, schools, not, created, ?]",7,"{'answer_start': [], 'text': []}"
99998,"Higher education, also called tertiary, third ...",572b8770111d821400f38ee8,What was another name used for Higher Education?,Education,[30],"[tertiary, third stage, or postsecondary educa...","[Higher, education, ,, also, called, tertiary,...",88,"[What, was, another, name, used, for, Higher, ...",9,"{'answer_start': [30], 'text': ['tertiary, thi..."


In [ ]:
# extract columns
df_squad_train = train_df[['answers','context','id','question']]
df_squad_validation = validation_df[['answers','context','id','question']]

In [ ]:
df_squad_train.to_json('/content/drive/MyDrive/ColabNotebooks/data/squad_train.jsonl',orient='records', lines=True)

In [ ]:
df_squad_validation.to_json('/content/drive/MyDrive/ColabNotebooks/data/squad_validation.jsonl',orient='records', lines=True)

Update dataset to match Natural Questions

In [ ]:
# add column indicating if question is answerable or non-answerable
# create function
def no_answer(start): 
  if start == []:
    return 0
  else:
    return 1

# apply to answer column
train_df["answer_yes_no"] =  train_df.apply(lambda x:no_answer(x['answers.answer_start']), axis=1)
validation_df["answer_yes_no"] =  validation_df.apply(lambda x:no_answer(x['answers.answer_start']), axis=1)

In [ ]:
# groupby unique context 
train_reduce_df = train_df.groupby('context').sample(n=1, random_state=1)
validation_reduce_df = validation_df.groupby('context').sample(n=1, random_state=1)

In [ ]:
# check the number of answerable to non_answerable for train dataset (percentage is 75%-25% ok)
train_reduce_df['answer_yes_no'].value_counts()

1    14373
0     4656
Name: answer_yes_no, dtype: int64

In [ ]:
# check the number of answerable to non_answerable for train dataset (percentage is nearly 50%-50%. Matches previous distribution ok)
validation_reduce_df['answer_yes_no'].value_counts()

1    611
0    593
Name: answer_yes_no, dtype: int64

In [ ]:
# further reduce the remaining training dataset to match the NQ dataset of ~ 8500
train_reduce_8500 = train_reduce_df.groupby('answer_yes_no').sample(frac=0.45, random_state=1)

In [ ]:
# check the number of answerable to non_answerable for train dataset (percentage is 75%-25% ok)
train_reduce_8500['answer_yes_no'].value_counts()

1    6468
0    2095
Name: answer_yes_no, dtype: int64

In [ ]:
# convert squad data back to seperate jsonl files

In [ ]:
# function to combine the start token and answer into dictionary
def to_dict(answer_start, answer): 
  dict = {}
  dict['answer_start'] = answer_start
  dict['text'] = answer
  return dict

In [ ]:
# run function
train_reduce_8500['answers'] = train_reduce_8500.apply(lambda x:to_dict(x['answers.answer_start'],x['answers.text']), axis=1)


In [ ]:
pd.set_option('display.max_colwidth', 50)
train_reduce_8500.head()

,context,id,question,title,answers.answer_start,answers.text,context_tokens,num_c_tokens,answer_yes_no,answers
10559,Texts on architecture have been written since ...,5acfa27f77cf76001a68565f,Who wrote canons in 2nd century BCE?,Architecture,[],[],"[Texts, on, architecture, have, been, written,...",53,0,"{'answer_start': [], 'text': []}"
69358,"According to his autobiography, Avicenna had m...",5ace99ba32bba1001ae4ac3d,What did Avicenna learn from the Shia scholar ...,Avicenna,[],[],"[According, to, his, autobiography, ,, Avicenn...",107,0,"{'answer_start': [], 'text': []}"
39235,The Virgin birth of Jesus was an almost univer...,5ad1620b645df0001a2d1936,In what century did Mary fulfill the prophecy?,Mary_(mother_of_Jesus),[],[],"[The, Virgin, birth, of, Jesus, was, an, almos...",183,0,"{'answer_start': [], 'text': []}"
55036,The early synapsid mammalian ancestors were sp...,5a39e1dc2f14dd001ac72628,The Sphenacodontia arose in the Paleogene and ...,Mammal,[],[],"[The, early, synapsid, mammalian, ancestors, w...",102,0,"{'answer_start': [], 'text': []}"
106783,"In a grounded-emitter transistor circuit, such...",5a7b6e4f21c2de001afea006,What changes the most in a grounded-emitter tr...,Transistor,[],[],"[In, a, grounded-emitter, transistor, circuit,...",102,0,"{'answer_start': [], 'text': []}"


In [ ]:
# extract columns
df_squad_train_reduce = train_reduce_8500[['answers','context','id','question']]

In [ ]:
df_squad_train_reduce.to_json('/content/drive/MyDrive/ColabNotebooks/data/squad-reduce-8500_train.jsonl',orient='records', lines=True)

Statistics for Reduced SQuAD dataset

In [ ]:
# create column for tokenized context
train_reduce_df["context_tokens"] = train_reduce_df['context'].apply(word_tokenize)

# create column for number of context tokens
train_reduce_df["num_c_tokens"] = [len(token) for token in train_reduce_df["context_tokens"]]

In [ ]:
# create column for tokenized question
train_reduce_df["question_tokens"] = train_reduce_df['question'].apply(word_tokenize)

# create column for number of question tokens
train_reduce_df["num_q_tokens"] = [len(token) for token in train_reduce_df["question_tokens"]]

In [ ]:
train_reduce_df.head()

,context,id,question,title,answers.answer_start,answers.text,answer_yes_no,context_tokens,num_c_tokens,question_tokens,num_q_tokens
60985,"""4 Minutes"" was released as the album's lead s...",5726e664dd62a815002e945c,4 minutes became Madonna's which number one si...,Madonna_(entertainer),[343],[thirteenth],1,"[``, 4, Minutes, '', was, released, as, the, a...",193,"[4, minutes, became, Madonna, 's, which, numbe...",13
74350,"""Alsatia"", the Latin form of Alsace's name, ha...",5727acde3acd2414000de95f,Which nationalities still refer it to Alslace ...,Alsace,[420],[English and Australian],1,"[``, Alsatia, '', ,, the, Latin, form, of, Als...",146,"[Which, nationalities, still, refer, it, to, A...",15
84425,"""At 11 am Petrus Canisius Van Lierde as Papal ...",5a6116aae9e1cc001a33cecf,When did Van Lierde give everyone a last farew...,Pope_John_XXIII,[],[],0,"[``, At, 11, am, Petrus, Canisius, Van, Lierde...",138,"[When, did, Van, Lierde, give, everyone, a, la...",10
55482,"""Azure, three ships with three masts, rigged a...",5a847dab7cf838001a46a86f,What does DEO DUCENTE NIL NOCET mean in Russian?,East_India_Company,[],[],0,"[``, Azure, ,, three, ships, with, three, mast...",153,"[What, does, DEO, DUCENTE, NIL, NOCET, mean, i...",10
53601,"""Baseball's Sad Lexicon,"" also known as ""Tinke...",57268b24708984140094c998,"Who wrote ""Baseball's Sad Lexicon""?",Chicago_Cubs,[114],[Franklin Pierce Adams],1,"[``, Baseball, 's, Sad, Lexicon, ,, '', also, ...",124,"[Who, wrote, ``, Baseball, 's, Sad, Lexicon, '...",9


In [ ]:
# number of questions
n_question = len(train_reduce_df['context'])
print("Number of questions :", n_question)

Number of questions : 19029


In [ ]:
# number of unique contexts
unique_contexts = len(pd.unique(train_reduce_df['context']))
print("Number of unique contexts :", unique_contexts)

Number of unique contexts : 19029


In [ ]:
# average number of tokens per question
q_avg = train_reduce_df['num_q_tokens'].mean(axis = 0) 
print("Average number of tokens per question :", q_avg)

Average number of tokens per question : 11.160964843134163


In [ ]:
# average number of tokens per context
c_unique = train_reduce_df.drop_duplicates(subset = ['context'])
c_avg = c_unique['num_c_tokens'].mean(axis = 0) 
print("Average number of tokens per context :", c_avg)

Average number of tokens per context : 134.1707393977613


In [ ]:
# get the number of answerable questions
drop_no_answer = train_reduce_df[~train_reduce_df['answers.answer_start'].str.len().eq(0)]
num_ans_question = len(drop_no_answer['context'])
per_ans_question = (num_ans_question / n_question) * 100
print("Number of answerable questions :", num_ans_question)
print("Percentage of answerable questions :", per_ans_question)

Number of answerable questions : 14373
Percentage of answerable questions : 75.532082610752


In [ ]:
# convert squad data back to seperate jsonl files

In [ ]:
# function to combine the start token and answer into dictionary
def to_dict(answer_start, answer): 
  dict = {}
  dict['answer_start'] = answer_start
  dict['text'] = answer
  return dict

In [ ]:
# run function
train_reduce_df['answers'] = train_reduce_df.apply(lambda x:to_dict(x['answers.answer_start'],x['answers.text']), axis=1)
validation_reduce_df['answers'] = validation_reduce_df.apply(lambda x:to_dict(x['answers.answer_start'],x['answers.text']), axis=1)

In [ ]:
pd.set_option('display.max_colwidth', 50)
train_reduce_df.head()

,context,id,question,title,answers.answer_start,answers.text,answer_yes_no,context_tokens,num_c_tokens,question_tokens,num_q_tokens,answers
60985,"""4 Minutes"" was released as the album's lead s...",5726e664dd62a815002e945c,4 minutes became Madonna's which number one si...,Madonna_(entertainer),[343],[thirteenth],1,"[``, 4, Minutes, '', was, released, as, the, a...",193,"[4, minutes, became, Madonna, 's, which, numbe...",13,"{'answer_start': [343], 'text': ['thirteenth']}"
74350,"""Alsatia"", the Latin form of Alsace's name, ha...",5727acde3acd2414000de95f,Which nationalities still refer it to Alslace ...,Alsace,[420],[English and Australian],1,"[``, Alsatia, '', ,, the, Latin, form, of, Als...",146,"[Which, nationalities, still, refer, it, to, A...",15,"{'answer_start': [420], 'text': ['English and ..."
84425,"""At 11 am Petrus Canisius Van Lierde as Papal ...",5a6116aae9e1cc001a33cecf,When did Van Lierde give everyone a last farew...,Pope_John_XXIII,[],[],0,"[``, At, 11, am, Petrus, Canisius, Van, Lierde...",138,"[When, did, Van, Lierde, give, everyone, a, la...",10,"{'answer_start': [], 'text': []}"
55482,"""Azure, three ships with three masts, rigged a...",5a847dab7cf838001a46a86f,What does DEO DUCENTE NIL NOCET mean in Russian?,East_India_Company,[],[],0,"[``, Azure, ,, three, ships, with, three, mast...",153,"[What, does, DEO, DUCENTE, NIL, NOCET, mean, i...",10,"{'answer_start': [], 'text': []}"
53601,"""Baseball's Sad Lexicon,"" also known as ""Tinke...",57268b24708984140094c998,"Who wrote ""Baseball's Sad Lexicon""?",Chicago_Cubs,[114],[Franklin Pierce Adams],1,"[``, Baseball, 's, Sad, Lexicon, ,, '', also, ...",124,"[Who, wrote, ``, Baseball, 's, Sad, Lexicon, '...",9,"{'answer_start': [114], 'text': ['Franklin Pie..."


In [ ]:
# extract columns
df_squad_train_reduce = train_reduce_df[['answers','context','id','question']]
df_squad_validation_reduce = validation_reduce_df[['answers','context','id','question']]

In [ ]:
df_squad_train_reduce.to_json('/content/drive/MyDrive/ColabNotebooks/data/squad-reduce_train.jsonl',orient='records', lines=True)

In [ ]:
df_squad_validation_reduce.to_json('/content/drive/MyDrive/ColabNotebooks/data/squad-reduce_validation.jsonl',orient='records', lines=True)